ND

In [1]:
import os
import sys
project_path = os.path.abspath(os.path.join(os.getcwd(), '..'))  # Parent of mltools
if project_path not in sys.path:
    sys.path.insert(0, project_path)


import numpy as np
import copy
import yaml
yaml.Dumper.ignore_aliases = lambda *args : True
import utils
import matplotlib.pyplot as plt
import tqdm
import glob


import importlib
importlib.reload(utils)

<module 'utils' from '/data2/jiawei/llm-understanding/concept-learning/utils.py'>

In [2]:
def get_ntr_ntest(ns,n_per=128,method="fid_axis"):
    indsgrid=np.stack(np.meshgrid(*([np.arange(n) for n in ns]),indexing="ij"),axis=-1)
    if method=="fid_axis":
        n_nonzero=np.sum(indsgrid!=0,axis=-1)
        sel=(n_nonzero<=1).flatten().astype(int)
        ntr=(sel*n_per).tolist()
        ntest=((1-sel)*n_per).tolist()
    else:
        raise ValueError("Unknown method")
    return ntr,ntest

In [3]:
ntr,ntest=get_ntr_ntest([2,2,2],n_per=2048,method="fid_axis")
ntr_gen,ntest_gen=get_ntr_ntest([2,2,2],n_per=5000,method="fid_axis")
np.sum(ntr),np.sum(ntest),np.sum(ntr_gen),np.sum(ntest_gen)

(np.int64(8192), np.int64(8192), np.int64(20000), np.int64(20000))

In [4]:
config={
    'experiment_directory': None,
    'seed': None,
    "dataset":"images_1",
    'n_samples_train':[],
    'n_samples_test':[],
    'n_samples_train_gen':[],
    'n_samples_test_gen':[],
    "data_params":{
        "n_classes":8,
        "image_size": 28,
        "noise_level": 0.001,
        "comp_dims":{"shape":2,"x":None,"y":None,"color":2,"size":2,"bg_color":None},
        
        "shape":{"names":["circle","triangle"]},
        "x":{"min":0.0,"max":0.0,"n":None,"s":0.0,"s_n":0.0},
        "y":{"min":0.0,"max":0.0,"n":None,"s":0.0,"s_n":0.0},
        "color":{"means":[[0.9,0.1,0.1],[0.1,0.1,0.9]],"mins":[[0.8,0.0,0.0],[0.0,0.0,0.8]],"maxs":[[1.0,0.2,0.2],[0.2,0.2,1.0]],"s_n":0.0},
        "size":{"means":[0.6,0.4],"mins":[0.55,0.4],"maxs":[0.65,0.45],"s_n":0.00,"min":0.1},
        "bg_color":{"means":[[1.0,1.0,1.0],[0.0,0.0,0.0]],"mins":[[1.0,1.0,1.0],[0.0,0.0,0.0]],"maxs":[[1.0,1.0,1.0],[0.0,0.0,0.0]],"s_n":0.0},
    },
    "model_params":{
        "model_type": "VDiff",
        "optimizer_type": "AdamW",
        "optimizer_params": {"lr": 0.001,"weight_decay": 0.01},
        "data_noise":1e-3,
        "beta_settings": {"type":"logsnr","noise_schedule":"learned_linear","gamma_min":-5., "gamma_max":10.},
        "network_params": {"mid_attn":True,
                           "v_conditioning_type":"sep_mlp",
                           "v_conditioning_dims":[11],
                           "chs": [64,128,256],
                           "num_res_blocks":2,
                           "init_scale":1.0},
    },
    "batch_size": 64,
    "train_ratio": 0.85,
    'num_steps': 40_000,
    'save_steps_start': 50,
    'save_steps': 120,

    "fig_n_col": 4,
}

In [5]:
configs_sc2ss2sb2=[]
for seed in [0,100,200,300,400]:
    fol=f"images_1/2x2x2_final/sc2ss2sb2_re/shape_color_size"
    experiment_name=f"seed={seed}"
    
    config_=copy.deepcopy(config)
    config_["seed"]=seed

    config_['n_samples_train']=ntr
    config_['n_samples_test']=ntest
    config_['n_samples_train_gen']=ntr_gen
    config_['n_samples_test_gen']=ntest_gen


    config_=utils.process_config(config_,check_only=True)
    experiment_directory=os.path.join("data",fol,experiment_name)
    config_["experiment_directory"]=experiment_directory
    yaml_fol=os.path.join("yamls",fol)
    os.makedirs(yaml_fol,exist_ok=True)
    yaml_path=os.path.join(yaml_fol,experiment_name+".yaml")
    yaml.dump(config_,open(yaml_path,"w"))
    configs_sc2ss2sb2.append(config_)

In [6]:
yaml_files=glob.glob("./yamls/images_1/2x2x2_final/sc2ss2sb2_re/*/*.yaml")

n_proc=5
chunked=[yaml_files[i::n_proc] for i in range(n_proc)]
for i,chunk in enumerate(chunked):
    with open(f"./yamls/images_1/2x2x2_final/sc2ss2sb2_re/yamls_{i}.txt","w") as f:
        for path in chunk:
            path=os.path.abspath(path)
            f.write(path+"\n")